In [1]:
import os
import json
import cv2
import numpy as np
from tqdm import tqdm

In [2]:
import random
SEED=42
random.seed(SEED)
np.random.seed(SEED)

BASE_IMG_DIR="../../data/raw/03_AIHUB_dataset/images"
BASE_JSON_DIR="../../data/raw/03_AIHUB_dataset/labeling"
SAVE_DIR="../../data/processed/03_AIHUB_dataset/cropped_data"

# 크기 설정 (w, h)
IMG_SIZE=(80, 40)

# 파일명에 따라 라벨 결정
def get_label_from_filename(filename):
    if "정상주시" in filename:
        return "open"
    elif "졸음재현" in filename or "하품재현" in filename:
        return "closed"
    else:
        return None

In [3]:
def process_all_images():
    for folder in tqdm(os.listdir(BASE_IMG_DIR), desc="Processing folders"):
        folder_img_path=os.path.join(BASE_IMG_DIR, folder)
        folder_json_path=os.path.join(BASE_JSON_DIR, folder)
        
        if not os.path.isdir(folder_img_path):
            continue

        for filename in os.listdir(folder_img_path):
            if not filename.endswith(".jpg"):
                continue

            label=get_label_from_filename(filename)
            if label is None:
                continue

            img_path=os.path.join(folder_img_path, filename)
            json_path=os.path.join(folder_json_path, filename.replace(".jpg", ".json"))

            if not os.path.exists(json_path):
                continue

            img=cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            if img is None:
                continue

            with open(json_path, "r") as f:
                data=json.load(f)

            try:
                leye_box=data["ObjectInfo"]["BoundingBox"]["Leye"]["Position"]
                reye_box=data["ObjectInfo"]["BoundingBox"]["Reye"]["Position"]

                for LorR, bbox in [("left", leye_box), ("right", reye_box)]:
                    x1, y1, x2, y2=bbox
                    eye_crop=img[y1:y2, x1:x2]

                    if eye_crop is None or eye_crop.size==0:
                        continue

                    eye_resized=cv2.resize(eye_crop, IMG_SIZE)

                    save_dir=os.path.join(SAVE_DIR, label)
                    os.makedirs(save_dir, exist_ok=True)

                    unique_name=f"{folder}_{filename.replace('.jpg','')}_{LorR}.jpg"
                    save_path=os.path.join(save_dir, unique_name)

                    cv2.imwrite(save_path, eye_resized)

            except KeyError:
                continue

In [4]:
process_all_images()

Processing folders:   0%|          | 0/243 [00:00<?, ?it/s]

Processing folders: 100%|██████████| 243/243 [03:30<00:00,  1.15it/s]


In [5]:
from collections import Counter
import os

base_dir="../../data/processed/03_AIHUB_dataset/cropped_data"
open_cnt=len(os.listdir(os.path.join(base_dir, "open")))
closed_cnt=len(os.listdir(os.path.join(base_dir, "closed")))

print("Open:", open_cnt)
print("Closed:", closed_cnt)

Open: 95213
Closed: 69549
